In [2]:
import time
notebookstart= time.time()
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os
import gc
from sklearn import preprocessing
from sklearn.model_selection import KFold
from sklearn import preprocessing
from sklearn.metrics import roc_auc_score, roc_curve

from rgf.sklearn import RGFClassifier


In [3]:
train = pd.read_csv('/media/limbo/Home-Credit/data/application_train.csv.zip')
test = pd.read_csv('/media/limbo/Home-Credit/data/application_test.csv')
y = train['TARGET']

n_train = train.shape[0]

In [4]:
d9 = pd.read_csv('../data/ds9.csv', header=0, index_col=None)

In [5]:
d9.head()

,SK_ID_CURR,external_sources_mean,STCK_PREV_APP_ALL_,STCK_BER_ALL_,external_sources_nanmedian,external_sources_sum,external_sources_min,external_sources_max,AMT_CREDIT_INSTALLMENTS,NAME_EDUCATION_TYPE_CODE_GENDER_EXT_SOURCE_3_min_diff,...,x_5,x_6,x_7,x_8,x_9,x_10,retirement_age,long_employment,EXT_SOURCE_1,EXT_SOURCE_2
0,100002,0.161787,0.133842,0.084131,0.139376,0.485361,0.083037,0.262949,16.461104,0.138849,...,48,35,140,70,450,13,1,1,0.083037,0.262949
1,100003,0.466757,0.040167,0.061941,0.466757,0.933513,0.311267,0.622246,36.234085,NaN,...,35,124,60,209,168,821,0,1,0.311267,0.622246
2,100004,0.642739,0.117715,0.068417,0.642739,1.285479,0.555912,0.729567,20.000000,0.729039,...,60,112,107,128,73,114,0,1,NaN,0.555912
3,100006,0.650442,0.082768,NaN,0.650442,0.650442,0.650442,0.650442,10.532818,NaN,...,61,81,85,164,214,1016,0,0,NaN,0.650442
4,100007,0.322738,0.090020,0.065612,0.322738,0.322738,0.322738,0.322738,23.461618,NaN,...,59,15,21,193,32,612,0,0,NaN,0.322738


In [6]:
no_hc = pd.read_csv('../data/no_hc_loan_cust.csv', index_col=None, header=0)

In [7]:
ind = d9['SK_ID_CURR'].isin(no_hc.values[:, 0])

In [8]:
for f in d9.columns:
    if 'ex' in f:
        d9.loc[ind.values, f] = np.nan
    if 'EX' in f:
        d9.loc[ind.values, f] = np.nan
    

external_sources_mean
external_sources_nanmedian
external_sources_sum
external_sources_min
external_sources_max
NAME_EDUCATION_TYPE_CODE_GENDER_EXT_SOURCE_3_min_diff
NAME_FAMILY_STATUS_NAME_EDUCATION_TYPE_EXT_SOURCE_3_mean_diff
NAME_FAMILY_STATUS_NAME_EDUCATION_TYPE_EXT_SOURCE_3_max_abs_diff
NAME_FAMILY_STATUS_CODE_GENDER_EXT_SOURCE_3_mean_diff
NAME_FAMILY_STATUS_CODE_GENDER_EXT_SOURCE_2_min_diff
NAME_EDUCATION_TYPE_CODE_GENDER_EXT_SOURCE_2_max_abs_diff
NAME_EDUCATION_TYPE_CODE_GENDER_EXT_SOURCE_3_max_abs_diff
EXT_SOURCE_3.x
NAME_FAMILY_STATUS_CODE_GENDER_EXT_SOURCE_2_min_abs_diff
NAME_EDUCATION_TYPE_CODE_GENDER_EXT_SOURCE_2_max_diff
NAME_FAMILY_STATUS_CODE_GENDER_EXT_SOURCE_2_max_diff
NAME_EDUCATION_TYPE_CODE_GENDER_EXT_SOURCE_2_min_diff
NAME_FAMILY_STATUS_CODE_GENDER_EXT_SOURCE_3_max_abs_diff
NAME_FAMILY_STATUS_NAME_EDUCATION_TYPE_EXT_SOURCE_3_min_diff
NAME_FAMILY_STATUS_NAME_EDUCATION_TYPE_EXT_SOURCE_3_min_abs_diff
EXT_1_DIV_BIRTH_SURESH.x
NAME_FAMILY_STATUS_NAME_EDUCATION_TYPE_EXT_

In [9]:
d9.head()

,SK_ID_CURR,external_sources_mean,STCK_PREV_APP_ALL_,STCK_BER_ALL_,external_sources_nanmedian,external_sources_sum,external_sources_min,external_sources_max,AMT_CREDIT_INSTALLMENTS,NAME_EDUCATION_TYPE_CODE_GENDER_EXT_SOURCE_3_min_diff,...,x_5,x_6,x_7,x_8,x_9,x_10,retirement_age,long_employment,EXT_SOURCE_1,EXT_SOURCE_2
0,100002,0.161787,0.133842,0.084131,0.139376,0.485361,0.083037,0.262949,16.461104,0.138849,...,48,35,140,70,450,13,1,1,0.083037,0.262949
1,100003,0.466757,0.040167,0.061941,0.466757,0.933513,0.311267,0.622246,36.234085,NaN,...,35,124,60,209,168,821,0,1,0.311267,0.622246
2,100004,0.642739,0.117715,0.068417,0.642739,1.285479,0.555912,0.729567,20.000000,0.729039,...,60,112,107,128,73,114,0,1,NaN,0.555912
3,100006,0.650442,0.082768,NaN,0.650442,0.650442,0.650442,0.650442,10.532818,NaN,...,61,81,85,164,214,1016,0,0,NaN,0.650442
4,100007,0.322738,0.090020,0.065612,0.322738,0.322738,0.322738,0.322738,23.461618,NaN,...,59,15,21,193,32,612,0,0,NaN,0.322738


In [10]:
del d9['TARGET.x']

In [11]:
test_file_path = "test_rgf_big_0.csv"
validation_file_path = 'oof_rgf_big_0.csv'
num_folds = 5

In [ ]:
gc.collect()
train_df = d9.iloc[0:n_train]
test_df = d9.iloc[n_train:]

val_df = train[['SK_ID_CURR', 'TARGET']].copy()

print("Starting LightGBM. Train shape: {}, test shape: {}".format(train_df.shape, test_df.shape))
gc.collect()
# Cross validation model
folds = KFold(n_splits=num_folds, shuffle=True, random_state=1001)
# Create arrays and dataframes to store results
oof_preds = np.zeros(train_df.shape[0])
sub_preds = np.zeros(test_df.shape[0])
feature_importance_df = pd.DataFrame()

feats = [f for f in train_df.columns if f not in ['TARGET','SK_ID_CURR','SK_ID_BUREAU','SK_ID_PREV','index']]

x_train = train_df[feats].values
x_test = test_df[feats].values

print(train_df.shape, x_train.shape)


for n_fold, (train_idx, valid_idx) in enumerate(folds.split(x_train, train['TARGET'])):
    model = RGFClassifier(max_leaf=1666,
                    algorithm="RGF_Sib",
                    test_interval=50,
                    l2=0.1,
                    n_iter=40,
                    reg_depth=2,
                    calc_prob='softmax',
                    learning_rate=0.5,
                    verbose=False)
    model.fit(x_train[train_idx], y_train[train_idx])
    oof_preds[valid_idx] = model.predict_proba(x_train[valid_idx])[:, 1]
    sub_preds += model.predict_proba(x_test)[:, 1] / folds.n_splits

    print('Fold %2d AUC : %.6f' % (n_fold + 1, roc_auc_score(y_train[valid_idx], oof_preds[valid_idx])))
    gc.collect()



Starting LightGBM. Train shape: (307511, 2043), test shape: (48744, 2043)
(307511, 2043) (307511, 2042)


In [ ]:
print('Full AUC score %.6f' % roc_auc_score(train['TARGET'], oof_preds))
# Write submission file and plot feature importance

sub_df = test[['SK_ID_CURR']].copy()
sub_df['TARGET'] = sub_preds
sub_df[['SK_ID_CURR', 'TARGET']].to_csv(test_file_path, index= False)


val_df['TARGET'] = oof_preds
val_df[['SK_ID_CURR', 'TARGET']].to_csv(validation_file_path, index= False)

gc.collect()